<a href="https://colab.research.google.com/github/rwcitek/C11-capstone-project/blob/main/eBird/ebird_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with BigQuery


We'll see if this runs in CoLab.




### Set up the workspace


In [1]:
!rm -rf data/


In [4]:
mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
cd data

/content/data


In [6]:
pwd

'/content/data'

### Download some sample data

In [7]:
%%capture output
%%bash
apt-get update
apt-get install -y tree jq curl less

In [8]:
!curl -O 'https://ebird.org/downloads/samples/ebd-datafile-SAMPLE.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  455k  100  455k    0     0   813k      0 --:--:-- --:--:-- --:--:--  812k


In [9]:
!ls -la


total 464
drwxr-xr-x 2 root root   4096 Nov 22 06:23 .
drwxr-xr-x 1 root root   4096 Nov 22 06:22 ..
-rw-r--r-- 1 root root 466849 Nov 22 06:23 ebd-datafile-SAMPLE.zip


In [10]:
!unzip ebd-datafile-SAMPLE.zip

Archive:  ebd-datafile-SAMPLE.zip
   creating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
  inflating: __MACOSX/._ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._ebd_US-AL-101_202204_202204_relApr-2022.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/terms_of_use.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._terms_of_use.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/BCRCodes.txt  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._BCRCodes.txt  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: __MACOSX/ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/._eBird_Basic_Dataset_Metadata_v1.14.pdf  
  inflating: ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/IBACod

In [11]:
!ls -la


total 472
drwxr-xr-x 4 root root   4096 Nov 22 06:23 .
drwxr-xr-x 1 root root   4096 Nov 22 06:22 ..
-rw-r--r-- 1 root root 466849 Nov 22 06:23 ebd-datafile-SAMPLE.zip
drwxr-xr-x 2 root root   4096 Jun  8  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE
drwxr-xr-x 3 root root   4096 Nov 22 06:23 __MACOSX


In [12]:
!rm -rf __MACOSX/

In [13]:
!tree ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/


ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/
├── BCRCodes.txt
├── ebd_US-AL-101_202204_202204_relApr-2022.txt
├── eBird_Basic_Dataset_Metadata_v1.14.pdf
├── IBACodes.txt
├── recommended_citation.txt
├── terms_of_use.txt
└── USFWSCodes.txt

0 directories, 7 files


In [14]:
%%bash
wc -l ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


1400 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


## Set up Google Cloud Platform

In [15]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [16]:
import os
project_id = "default-256400"
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id


## Need to create a *bucket*

Go to https://console.cloud.google.com/storage/browser




Got a bucket:

- https://console.cloud.google.com/storage/browser/c11-ebird;tab=objects?project=default-256400&prefix=&forceOnObjectsSortingFiltering=false

## Phase 1:



1. upload the CSV file to GS
1. import into BQ



In [17]:
file = 'ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt'


In [ ]:
!gcloud storage cp {file} gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [41]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


### From web interface

Create a data set and import CSV to data set:

- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

### Repeat using bq command

In [ ]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_2

In [ ]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_2 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt


Waiting on bqjob_r2c10e32fbe8bf2d0_0000018bf481da69_1 ... (1s) Current status: DONE   


In [ ]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_2   TABLE                                                  


## Phase 2:


1. compress the CSV file with gzip
1. upload the compressed CSV file to GS
1. import into BQ



In [ ]:
!ls -la {file}

-rw-r--r-- 1 root root 504698 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt


In [20]:
!gzip {file}

In [21]:
!ls -la {file}.gz


-rw-r--r-- 1 root root 60011 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [ ]:
!gcloud storage cp {file}.gz gs://c11-ebird/

Copying file://ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [ ]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


In [ ]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [ ]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_3 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Waiting on bqjob_r610c656fefcded4c_0000018bf48bc0f5_1 ... (1s) Current status: DONE   


In [ ]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_3   TABLE                                                  


In [ ]:
!bq rm -f {project_id}:ebird.ebird_small_3

In [ ]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


## Phase 3:


1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. untar and stream the compressed CSV file to GS
1. import into BQ



In [28]:
!ls -la {file}.gz

-rw-r--r-- 1 root root 60011 May 19  2022 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Create the tar file

In [35]:
path, file1 = file.split("/")

In [38]:
!tar -cvf example.tar -C {path} {file1}.gz

ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


In [39]:
!ls -la


total 544
drwxr-xr-x 3 root root   4096 Nov 22 06:37 .
drwxr-xr-x 1 root root   4096 Nov 22 06:22 ..
-rw-r--r-- 1 root root 466849 Nov 22 06:23 ebd-datafile-SAMPLE.zip
drwxr-xr-x 2 root root   4096 Nov 22 06:25 ebd_US-AL-101_202204_202204_relApr-2022_SAMPLE
-rw-r--r-- 1 root root  71680 Nov 22 06:44 example.tar


In [40]:
!tar -tvf example.tar

-rw-r--r-- root/root     60011 2022-05-19 17:35 ebd_US-AL-101_202204_202204_relApr-2022.txt.gz


Untar and stream to GS.

In [42]:
!gcloud storage ls -l gs://c11-ebird/


    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
TOTAL: 2 objects, 564709 bytes (551.47kiB)


In [43]:
%%bash
tar -O -xf example.tar ebd_US-AL-101_202204_202204_relApr-2022.txt.gz |
gcloud storage cp - gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz



Copying file://- to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz
  
......


In [44]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
     60011  2023-11-22T06:51:04Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz
TOTAL: 3 objects, 624720 bytes (610.08kiB)


In [45]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [46]:
!bq rm -f {project_id}:ebird.ebird_small_4

In [47]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_4 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz


Waiting on bqjob_r80dce339e5bf06_0000018bf5cd9540_1 ... (1s) Current status: DONE   


In [48]:
!bq ls {project_id}:ebird

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  ebird_small     TABLE                                                  
  ebird_small_4   TABLE                                                  


In [ ]:
!bq show {project_id}:ebird.ebird_small

In [ ]:
!bq show {project_id}:ebird.ebird_small_4

In [51]:
!bq rm -f {project_id}:ebird.ebird_small_4

## Phase 4:



1. compress the CSV file with gzip
1. create a tar file with the compressed CSV file
1. upload the tar file to DO's object store
1. get, untar, and stream the compressed CSV file to GS
1. import into BQ


In [52]:
!curl -I https://cnmi-ebird.sfo3.digitaloceanspaces.com/example.tar

HTTP/2 200 
content-length: 71680
accept-ranges: bytes
last-modified: Wed, 22 Nov 2023 06:55:56 GMT
x-rgw-object-type: Normal
etag: "fb5a9186a43fc4d150feb7bfff027363"
x-amz-request-id: tx000004c7bf6cb0b208cba-00655da61b-3c6f4933-sfo3a
content-type: application/x-tar
date: Wed, 22 Nov 2023 06:56:27 GMT
vary: Origin, Access-Control-Request-Headers, Access-Control-Request-Method
strict-transport-security: max-age=15552000; includeSubDomains; preload
x-envoy-upstream-healthchecked-cluster: 



In [57]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/example.tar |
tar -O -xf - ebd_US-AL-101_202204_202204_relApr-2022.txt.gz |
dd bs=1M status=progress |
gcloud storage cp - gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz


0+3 records in
0+3 records out
60011 bytes (60 kB, 59 KiB) copied, 0.528014 s, 114 kB/s
Copying file://- to gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
  
.....


11100700672 bytes (11 GB, 10 GiB) copied, 25 s, 444 MB/s
0+1751059 records in
0+1751059 records out
11479859200 bytes (11 GB, 11 GiB) copied, 25.8381 s, 444 MB/s
^C


In [58]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
     60011  2023-11-22T00:57:52Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz
     60011  2023-11-22T06:51:04Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz
     60011  2023-11-22T07:04:01Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz
TOTAL: 4 objects, 684731 bytes (668.68kiB)


In [59]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [60]:
!bq rm -f {project_id}:ebird.ebird_small_5

In [61]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebird_small_5 \
  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz


Waiting on bqjob_r5d684cc5be8246f8_0000018bf5d99405_1 ... (1s) Current status: DONE   


In [69]:
!bq ls {project_id}:ebird

    tableId     Type    Labels   Time Partitioning   Clustered Fields  
 ------------- ------- -------- ------------------- ------------------ 
  ebird_small   TABLE                                                  


In [63]:
!bq show {project_id}:ebird.ebird_small

Table default-256400:ebird.ebird_small

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  21 Nov 23:41:57   |- GLOBAL_UNIQUE_IDENTIFIER: string     1399         534727                                                            534727                41452                          
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                             

In [64]:
!bq show {project_id}:ebird.ebird_small_5

Table default-256400:ebird.ebird_small_5

   Last modified                   Schema                   Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  22 Nov 07:06:00   |- GLOBAL_UNIQUE_IDENTIFIER: string     1399         534727                                                            534727                                               
                    |- LAST_EDITED_DATE: timestamp                                                                                                                                              
                    |- TAXONOMIC_ORDER: integer                                                                                                                                           

In [65]:
!bq rm -f {project_id}:ebird.ebird_small_5

In [68]:
!gcloud storage rm gs://c11-ebird/*.gz

Removing objects:
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.gz...
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v02.gz...
Removing gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt.v03.gz...


## Transfer 5GB eBird file

In [74]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/ebd_sampling_relSep-2023.tar |
dd bs=1M status=progress |
tar -tvf -


-rw-r--r-- ebdBuilder/ebdBuilder 4815040900 2023-10-15 18:45 ebd_sampling_relSep-2023.txt.gz
-rwxr-xr-x ebdBuilder/ebdBuilder       1960 2023-10-15 10:00 BCRCodes.txt
-rwxr-xr-x ebdBuilder/ebdBuilder     126957 2023-10-15 10:00 IBACodes.txt
-rwxr-xr-x ebdBuilder/ebdBuilder      39670 2023-10-15 10:00 USFWSCodes.txt
-rw-r--r-- ebdBuilder/ebdBuilder     431439 2023-10-15 10:00 eBird_Basic_Dataset_Metadata_v1.15.pdf
-rwxr-xr-x ebdBuilder/ebdBuilder        103 2023-10-15 10:00 recommended_citation.txt
-rwxr-xr-x ebdBuilder/ebdBuilder       6703 2023-10-15 10:00 terms_of_use.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4592M  100 4592M    0     0  46.4M      0  0:01:38  0:01:38 --:--:-- 46.9M

0+337897 records in
0+337897 records out
4815656960 bytes (4.8 GB, 4.5 GiB) copied, 98.8393 s, 48.7 MB/s


In [75]:
%%bash
curl -s https://cnmi-ebird.sfo3.digitaloceanspaces.com/ebd_sampling_relSep-2023.tar |
tar -O -xf - ebd_sampling_relSep-2023.txt.gz |
dd bs=1M status=progress |
gcloud storage cp - gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz


Copying file://- to gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
  
4780635648 bytes (4.8 GB, 4.5 GiB) copied, 253 s, 18.9 MB/s
0+326225 records in
0+326225 records out
4815040900 bytes (4.8 GB, 4.5 GiB) copied, 253.737 s, 19.0 MB/s
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [77]:
!gcloud storage ls -l gs://c11-ebird/

    504698  2023-11-21T23:28:26Z  gs://c11-ebird/ebd_US-AL-101_202204_202204_relApr-2022.txt
4815040900  2023-11-22T07:20:39Z  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz
TOTAL: 2 objects, 4815545598 bytes (4.48GiB)


In [78]:
%%bash
bq --project_id=${GOOGLE_CLOUD_PROJECT} \
  load \
  --source_format=CSV \
  --field_delimiter='\t' \
  --autodetect \
  --skip_leading_rows=1 \
  ebird.ebd_sampling_relSep_2023 \
  gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz


BigQuery error in load operation: Error processing job
'default-256400:bqjob_r17564e9819be84cd_0000018bf5eeb6ea_1': Error while reading
data, error message: Input CSV files are not splittable and at least one of the
files is larger than the maximum allowed size. Size is: 4815040900. Max allowed
size is: 4294967296. File: gs://c11-ebird/ebd_sampling_relSep-2023.txt.gz


Waiting on bqjob_r17564e9819be84cd_0000018bf5eeb6ea_1 ... (0s) Current status: DONE   


CalledProcessError: ignored